### 필요 라이브러리 로드

In [ ]:
import os
import openai  # `openai` 모듈을 직접 임포트
import json
import gradio as gr
import requests
import xml.etree.ElementTree as ET
import Jetson.GPIO as GPIO
import time
import math

In [ ]:
# 환경 변수에 API 키 설정
os.environ['OPENAI_API_KEY'] = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxx'

# OpenAI API 키 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

# 예시 API 호출 (모델 목록 가져오기)
response = openai.Model.list()
print(json.dumps(response, indent=2))

In [ ]:
# 에어코리아 API 설정
API_KEY = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
BASE_URL = "http://apis.data.go.kr/B552584/ArpltnInforInqireSvc"

# Gradio 챗봇
## 함수 정의


*   API 이용 대전의 미세먼지 값 로드 하는 함수: get_station_fine_dust(), get_all_fine_dust()
*   실시간 미세먼지 측정 로드 함수: measure_pm25()
*   사용자 메시지 처리 함수: process()



In [ ]:
# 에어 코리아에서 API를 통해 미세먼지(PM 2.5) 값 불러오기
# "http://apis.data.go.kr/B552584/ArpltnInforInqireSvc"

# 특정 도시의 미세먼지 데이터 가져오기
def get_station_fine_dust(city_name, station_name):
    endpoint = f"{BASE_URL}/getCtprvnRltmMesureDnsty"
    params = {
        "serviceKey": requests.utils.unquote(API_KEY),
        "returnType": "xml",
        "numOfRows": 100,
        "pageNo": 1,
        "sidoName": city_name,
        "ver": "1.3"
    }

    response = requests.get(endpoint, params=params)
    if response.status_code == 200:
        root = ET.fromstring(response.text)
        items = root.findall(".//item")
        for item in items:
            station = item.find("stationName").text
            if station == station_name:
                pm25_value = item.find("pm25Value").text
                return pm25_value
        return None
    else:
        return None

# 도시 중 대전 전체의 미세먼지 데이터 가져오기
def get_all_fine_dust(city_name):
    endpoint = f"{BASE_URL}/getCtprvnRltmMesureDnsty"
    params = {
        "serviceKey": requests.utils.unquote(API_KEY),
        "returnType": "xml",
        "numOfRows": 100,
        "pageNo": 1,
        "sidoName": city_name,
        "ver": "1.3"
    }

    response = requests.get(endpoint, params=params)
    if response.status_code == 200:
        root = ET.fromstring(response.text)
        items = root.findall(".//item")
        fine_dust_data = []
        for item in items:
            station_name = item.find("stationName").text
            pm25_value = item.find("pm25Value").text
            fine_dust_data.append(f"측정소: {station_name}, 미세먼지(PM2.5): {pm25_value}㎍/㎥")
        return "\n".join(fine_dust_data)
    else:
        return f"API 요청 실패: {response.status_code}"

In [ ]:
# 실시간 미세먼지 측정 (Jetson GPIO)
def measure_pm25(pin=8, sample_time_ms=10000):
    GPIO.setmode(GPIO.BCM)
    GPIO.setup(pin, GPIO.IN)
    low_pulse_occupancy = 0
    start_time = time.time()

    try:
        while (time.time() - start_time) * 1000 <= sample_time_ms:
            pulse_start = time.time()
            while GPIO.input(pin) == GPIO.LOW:
                pass
            pulse_end = time.time()
            low_pulse_occupancy += (pulse_end - pulse_start) * 1e6

        ratio = low_pulse_occupancy / (sample_time_ms * 10.0)
        concentration = (
            1.1 * 0.22 * math.pow(ratio, 3) -
            3.8 * 0.22 * math.pow(ratio, 2) +
            520 * 0.22 * ratio +
            0.62
        ) * 10**-4
        return round(concentration, 2)
    except Exception as e:
        return f"센서 오류: {e}"
    finally:
        GPIO.cleanup()

In [ ]:
# 사용자 메시지 처리
def process(user_message, chat_history, state):
    # 실시간 센서 측정 값
    sensor_pm25_value = measure_pm25()

    # 특정 동 추출용 리스트
    station_list = ["읍내동", "문평동", "문창동", "구성동", "노은동", "상대동(대전)", "관평동", "대흥동1", "성남동1", "대성동", "정림동", "둔산동", "월평동"]

    if "실시간" in user_message and "미세먼지" in user_message:
        # 실시간 측정 데이터만 반환
        fine_dust_info = f"실시간 측정된 미세먼지 농도는 {sensor_pm25_value}㎍/㎥입니다."
    elif "미세먼지" in user_message:
        station_name = None
        for s in station_list:
            if s in user_message:
                station_name = s
                break

        if station_name:
            station_pm25_str = get_station_fine_dust("대전", station_name)
            if station_pm25_str is None:
                fine_dust_info = f"'{station_name}'에 대한 데이터를 찾을 수 없습니다."
            else:
                # 문자열을 실수로 변환 (API 값이 숫자가 아닐 경우 예외 처리 필요)
                try:
                    station_pm25_value = float(station_pm25_str)
                    fine_dust_info = f"{station_name}의 미세먼지(PM2.5) 농도는 {station_pm25_value}㎍/㎥입니다.\n"

                    # 실시간 측정값과 비교
                    if isinstance(sensor_pm25_value, (int, float)):
                        diff = round(sensor_pm25_value - station_pm25_value, 2)
                        if diff > 0:
                            fine_dust_info += f"현재 실내 측정값이 더 높습니다. 미세먼지 농도의 차이값이 {diff}㎍/㎥ 차이가 나며 환기가 필요합니다."
                        else:
                            fine_dust_info += f"현재 실내 측정값이 더 낮거나 같습니다. 미세먼지 농도의 차이값은 {diff}㎍/㎥ 입니다."
                    else:
                        fine_dust_info += "실시간 센서 데이터에 오류가 있습니다."

                except ValueError:
                    fine_dust_info = f"{station_name} 측정값을 변환하는 중 오류가 발생했습니다."
        else:
            # 특정 동을 지정하지 않은 경우 대전 전체 데이터를 보여줌
            fine_dust_info = get_all_fine_dust("대전")
    else:
        fine_dust_info = "미세먼지 데이터 요청 또는 실시간 데이터를 입력해주세요."

    chat_history.append((user_message, fine_dust_info))
    return "", chat_history, state

In [ ]:
# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="미세먼지 챗봇")
    user_textbox = gr.Textbox(label="질문 입력")
    state = gr.State()

    user_textbox.submit(process, [user_textbox, chatbot, state], [user_textbox, chatbot, state])

demo.launch(share=True, debug=True)


# Discord 이용 알림


*   실내 미세먼지 농도가 기준치에 비해 높거나 낮으면 디스코드 앱을 통한 알림을 울리도록 함



In [ ]:
!pip install discord
!pip install python-dotenv

In [ ]:
import Jetson.GPIO as GPIO
import time
import math
import discord
from discord.ext import commands
import asyncio
import nest_asyncio
from dotenv import load_dotenv
import os

In [ ]:
# nest_asyncio를 적용하여 Jupyter Notebook의 이벤트 루프와 호환
nest_asyncio.apply()

# 환경 변수 로드
load_dotenv()

DISCORD_CHANNEL_ID = "xxxxxxxxxxxxxxx"  # 채널 ID
DISCORD_BOT_TOKEN = "xxxxxxxxxxxxxxxxxx"  # 봇 토큰

In [ ]:
# PM2.5 농도 측정 함수
def measure_pm25():
    """
    PM2.5 농도를 측정하여 반환하는 함수.
    """
    pin = 8
    sample_time_ms = 30000  # 30초 샘플링 시간

    # GPIO 초기화
    GPIO.setmode(GPIO.BCM)
    GPIO.setup(pin, GPIO.IN)

    low_pulse_occupancy = 0
    start_time = time.time()

    try:
        while (time.time() - start_time) * 1000 <= sample_time_ms:
            pulse_start = time.time()
            while GPIO.input(pin) == GPIO.LOW:
                pass
            pulse_end = time.time()

            # LOW 상태 지속 시간 계산
            pulse_duration = (pulse_end - pulse_start) * 1e6  # 마이크로초 단위
            low_pulse_occupancy += pulse_duration

        # PM2.5 농도 계산
        ratio = low_pulse_occupancy / (sample_time_ms * 10.0)
        concentration = (
            1.1*0.22* math.pow(ratio, 3) - 3.8 *0.22* math.pow(ratio, 2) + 520 *0.22* ratio + 0.62
        )*10**-4
        return round(concentration, 2)

    except Exception as e:
        print(f"Error during measurement: {e}")
        return None

    finally:
        GPIO.cleanup()

In [ ]:
# Discord 봇 설정
intents = discord.Intents.default()
intents.message_content = True
bot = commands.Bot(command_prefix='!', intents=intents)

@bot.event
async def on_ready():
    print(f'Bot is ready')
    channel = bot.get_channel(DISCORD_CHANNEL_ID)
    if channel is None:
        print('채널이 없어요 😂')
        return
    print(f'봇이 {channel}에서 준비되었습니다! 😊')
    bot.loop.create_task(pm25_alert(channel))  # PM2.5 알림 태스크 시작

# PM2.5 상태 확인 및 알림
async def pm25_alert(channel):
    while True:
        pm25_concentration = measure_pm25()
        if pm25_concentration is not None:
            if pm25_concentration > 20:
                warning_message = (
                    f"🚨 경고: PM2.5 농도가 {pm25_concentration}㎍/㎥로 위험 수준입니다!\n"
                    f"🌬️ 환기가 필요합니다."
                )
                await channel.send(warning_message)
            else:
                normal_message = (
                    f"✅ 정상: PM2.5 농도가 {pm25_concentration}㎍/㎥로 정상 수준입니다."
                )
                await channel.send(normal_message)
        else:
            await channel.send("❌ PM2.5 농도를 측정할 수 없습니다.")

        await asyncio.sleep(3600)  # 1시간 간격으로 알림 전송

# 봇 명령어로 측정값 확인
@bot.command()
async def check_pm25(ctx):
    pm25_concentration = measure_pm25()
    if pm25_concentration is not None:
        if pm25_concentration > 20:
            await ctx.send(
                f"🚨 경고: PM2.5 농도가 {pm25_concentration}㎍/㎥로 위험 수준입니다!\n!"
                f"🌬️ 환기가 필요합니다."
            )
        else:
            await ctx.send(
                f"✅ 정상: PM2.5 농도가 {pm25_concentration}㎍/㎥로 정상 수준입니다."
            )
    else:
        await ctx.send("❌ PM2.5 농도를 측정할 수 없습니다.")

bot.run(DISCORD_BOT_TOKEN)